# Seleção de variáveis

Neste notebook iremos estudar o dataset __houses__ com o objetivo de avaliar a interação de primeira e segunda ordem que as variáveis causam umas nas outras. Além disso, será aplicado uma seleção de variáveis por meio do método stepwise.

In [1]:
#Leitura e apresentacao do dataset
casas <- read.table("houses.dat", header=T)
head(casas)

case,taxes,beds,baths,new,price,size
1,3104,4,2,0,279.9,2048
2,1173,2,1,0,146.5,912
3,3076,4,2,0,237.7,1654
4,1608,3,2,0,200.0,2068
5,1454,3,3,0,159.9,1477
6,2997,3,2,1,499.9,3153


Ajuste de dois modelos com todos os parâmetros (exceto taxes) avaliando a interação de primeira e segunda ordem.

In [2]:
#Ajustando os modelos
modN1 = with(casas,glm(price ~ beds + baths + new + size, family = gaussian(link="identity")))
modN2 = with(casas,glm(price ~ (beds + baths + new + size)^2, family = gaussian(link="identity")))

#Mostrando dados do modelo
summary(modN1)
summary(modN2)


Call:
glm(formula = price ~ beds + baths + new + size, family = gaussian(link = "identity"))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-215.747   -30.833    -5.574    18.800   164.471  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -28.84922   27.26116  -1.058  0.29262    
beds         -8.20238   10.44984  -0.785  0.43445    
baths         5.27378   13.08017   0.403  0.68772    
new          54.56238   19.21489   2.840  0.00553 ** 
size          0.11812    0.01232   9.585 1.27e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 2943.411)

    Null deviance: 1015150  on 99  degrees of freedom
Residual deviance:  279624  on 95  degrees of freedom
AIC: 1089.4

Number of Fisher Scoring iterations: 2



Call:
glm(formula = price ~ (beds + baths + new + size)^2, family = gaussian(link = "identity"))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-138.303   -28.479    -3.314    18.240   179.437  

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.391e+02  6.662e+01   2.087 0.039727 *  
beds        -5.954e+01  3.330e+01  -1.788 0.077180 .  
baths        1.426e+01  4.703e+01   0.303 0.762432    
new          1.033e+02  1.065e+02   0.970 0.334441    
size        -1.948e-03  5.613e-02  -0.035 0.972395    
beds:baths   8.135e-01  1.750e+01   0.046 0.963028    
beds:new    -1.045e+01  4.017e+01  -0.260 0.795403    
beds:size    3.277e-02  1.661e-02   1.972 0.051690 .  
baths:new   -1.043e+02  5.181e+01  -2.014 0.047075 *  
baths:size  -3.083e-03  1.590e-02  -0.194 0.846715    
new:size     1.053e-01  3.069e-02   3.431 0.000915 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian 

Analisando o p-value de cada variável vemos que existem umas que são mais relevantes e outras não. Note no modelo 2 (modN2) que as interações de segunda ordem mais relevantes são __new:size e baths:size__. Isso nos demonstra que existe relações entre variáveis que devemos considerar em nossa modelagem. Por exemplo, o valor da casa ser nova aparenta afetar o seu tamanho.

Vamos testar se há igualdade dos desvios para ambos os modelos por meio do teste F. Isso nos demonstrará se os modelos são estatísticamente iguais ou não.

In [3]:
#Testando se os desvios são iguais
anova(modN1,modN2,test='F')

Resid. Df,Resid. Dev,Df,Deviance,F,Pr(>F)
95,279624.1,NA,NA,NA,NA
89,217916.4,6,61707.71,4.200377,0.0009127911


Pelo resultado do p-value vemos que deve-se rejeitar a hipótese nula de que ambos possuem desvios iguais. Sabendo que os modelos são diferentes iremos criar um novo que contenha as interações mais relevantes. Após esse passo iremos remove-las individualmente para verificar o seu efeito no modelo.

In [4]:
#Modelo com as interações mais relevantes
modN3 = with(casas,lm(price ~ beds + baths + new + size + baths*new + new*size))

#Descrição do modelo 3
summary(modN3)

#Removendo uma interação
summary(update(modN3,.~. - new:baths))

#Removendo outra interação
summary(update(modN3,.~. - new:size))


Call:
lm(formula = price ~ beds + baths + new + size + baths * new + 
    new * size)

Residuals:
     Min       1Q   Median       3Q      Max 
-177.127  -29.451   -3.575   16.559  184.867 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -28.24361   26.30667  -1.074 0.285767    
beds          -4.38277    9.96498  -0.440 0.661090    
baths         14.68847   12.83526   1.144 0.255401    
new           58.10608   81.21488   0.715 0.476117    
size           0.09863    0.01296   7.610 2.19e-11 ***
baths:new   -103.91647   47.76275  -2.176 0.032114 *  
new:size       0.10757    0.03022   3.560 0.000586 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 51.4 on 93 degrees of freedom
Multiple R-squared:  0.758,	Adjusted R-squared:  0.7424 
F-statistic: 48.55 on 6 and 93 DF,  p-value: < 2.2e-16



Call:
lm(formula = price ~ beds + baths + new + size + new:size)

Residuals:
     Min       1Q   Median       3Q      Max 
-181.045  -27.047   -5.318   14.777  189.677 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -19.80698   26.53101  -0.747  0.45719    
beds         -5.07973   10.15570  -0.500  0.61811    
baths         7.63252   12.66301   0.603  0.54813    
new         -80.07128   51.61507  -1.551  0.12419    
size          0.10326    0.01304   7.921 4.68e-12 ***
new:size      0.06173    0.02208   2.795  0.00628 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 52.41 on 94 degrees of freedom
Multiple R-squared:  0.7457,	Adjusted R-squared:  0.7322 
F-statistic: 55.13 on 5 and 94 DF,  p-value: < 2.2e-16



Call:
lm(formula = price ~ beds + baths + new + size + baths:new)

Residuals:
     Min       1Q   Median       3Q      Max 
-212.664  -31.048   -5.088   18.072  167.790 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -26.71176   27.88880  -0.958    0.341    
beds         -7.97357   10.51145  -0.759    0.450    
baths         4.52548   13.26817   0.341    0.734    
new          20.96484   85.39741   0.245    0.807    
size          0.11721    0.01258   9.319 5.16e-15 ***
baths:new    14.65982   36.29788   0.404    0.687    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 54.49 on 94 degrees of freedom
Multiple R-squared:  0.725,	Adjusted R-squared:  0.7104 
F-statistic: 49.57 on 5 and 94 DF,  p-value: < 2.2e-16


Analisando as 3 opções do terceiro modelo criado, note que o último indica que apenas size é uma variável relevante. Por isso iremos descartá-lo. Comparando o primeiro com o segundo notamos que o seu valor de adjusted R-squared são bem parecidos. Como o segundo consegue explicar bem os dados com menos parâmetros iremos escolhe-lo para continuar este notebook.

Dado a opção 2 do terceiro modelo vamos remover a variável baths e avaliar o efeito disso.

In [5]:
modN4 = update(modN3,.~. - new:baths - baths)
summary(modN4)


Call:
lm(formula = price ~ beds + new + size + new:size)

Residuals:
     Min       1Q   Median       3Q      Max 
-181.982  -28.088   -6.876   15.571  194.062 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -14.67865   25.04536  -0.586  0.55921    
beds         -3.81395    9.90285  -0.385  0.70100    
new         -77.65111   51.28592  -1.514  0.13332    
size          0.10694    0.01148   9.316 4.78e-15 ***
new:size      0.06084    0.02196   2.771  0.00673 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 52.23 on 95 degrees of freedom
Multiple R-squared:  0.7447,	Adjusted R-squared:  0.734 
F-statistic: 69.28 on 4 and 95 DF,  p-value: < 2.2e-16
